In [1]:
from neo4j import GraphDatabase
driver = GraphDatabase.driver('bolt://neo4j:7687', auth=("neo4j", "BDII2023"))
session = driver.session()
def run_query(query, return_data=True, **kwargs):
        results = session.run(query, kwargs)
        if return_data:
            return results.data()
        return results

### 9. Calcular el total de oro que valen los tesoros de una mazmorra.

In [2]:
mazmorra = "Bentclock, Laboratory of the Sexy Thieves"
query = """
MATCH (r:Room{dungeon_name: $mazmorra})-[c:CONTAINS]->(l:Loot)
RETURN SUM(c.amount * l.gold) as total_gold
"""
results = run_query(query, mazmorra=mazmorra)
results

[{'total_gold': 720.0}]

### 10. Calcular el nivel medio de los monstruos de una mazmorra.


In [3]:
mazmorra = "Bentclock, Laboratory of the Sexy Thieves"
query = """
MATCH (r:Room{dungeon_name: $mazmorra})-[c:CONTAINS]->(m:Monster)
RETURN AVG(m.level * c.amount) as mean_level
"""
results = run_query(query, mazmorra=mazmorra)
results

[{'mean_level': 8.56637168141593}]

### 11. Calcular el número medio de pasillos (tanto entrantes como salientes) que tienen las salas de una mazmorra.


In [10]:
mazmorra = "Bentclock, Laboratory of the Sexy Thieves"
query = """
MATCH (r1: Room{dungeon_name: $mazmorra})-[i: IS_CONNECTED]-(r2: Room{dungeon_name: $mazmorra})
WITH r1, COUNT(i) as degree
RETURN AVG(degree) as mean_degree
"""
results = run_query(query, mazmorra=mazmorra)
results

[{'mean_degree': 4.271186440677967}]

### 12. Buscar la/las salas que contienen el/los monstruos de más nivel de la mazmorra.


In [39]:
mazmorra = "Bentclock, Laboratory of the Sexy Thieves"
query = """
MATCH (r: Room{dungeon_name: $mazmorra})-[:CONTAINS]->(m:Monster)
WITH MAX(m.level) as max_level
MATCH (r2:Room{dungeon_name: $mazmorra})-[:CONTAINS]->(m:Monster{level: max_level})
RETURN r2.room_name as room_name
"""
results = run_query(query, mazmorra=mazmorra)
results

[{'room_name': 'snobbish catacombs of otaku'},
 {'room_name': 'fancy gallery of presidents'}]

### 13. Calcular la experiencia total de cada uno de los encuentros (grupo de monstruos presentes en una sala) de una mazmorra y mostrarlos ordenados de mayor a menor experiencia.


In [42]:
mazmorra = "Bentclock, Laboratory of the Sexy Thieves"
query = """
MATCH (r: Room{dungeon_name: $mazmorra})-[c: CONTAINS]->(m: Monster)
RETURN r.room_name, SUM(c.amount * m.exp) as total_exp
ORDER BY total_exp DESC
"""
results = run_query(query, mazmorra=mazmorra)
results

[{'r.room_name': 'snobbish catacombs of otaku', 'total_exp': 5450},
 {'r.room_name': 'amorous bedroom ', 'total_exp': 5000},
 {'r.room_name': 'fancy gallery of presidents', 'total_exp': 5000},
 {'r.room_name': 'ballroom ', 'total_exp': 4350},
 {'r.room_name': 'atrium of sumo wrestlers', 'total_exp': 3850},
 {'r.room_name': 'larder of presidents', 'total_exp': 3450},
 {'r.room_name': 'bedroom of bandits', 'total_exp': 3400},
 {'r.room_name': 'laboratory ', 'total_exp': 3350},
 {'r.room_name': 'fancy larder ', 'total_exp': 2750},
 {'r.room_name': 'ancient observatory of stoners', 'total_exp': 2700},
 {'r.room_name': 'screeching barracks of nerds', 'total_exp': 2650},
 {'r.room_name': 'abnormal vault of bandits', 'total_exp': 2500},
 {'r.room_name': 'observatory ', 'total_exp': 2450},
 {'r.room_name': 'uptight storage room of hippies', 'total_exp': 2300},
 {'r.room_name': 'kitchen of unknowns', 'total_exp': 2300},
 {'r.room_name': 'worried sanctum ', 'total_exp': 2250},
 {'r.room_name': '

### 14. Buscar la sala dónde este el encuentro (grupo de monstruos presentes en una sala) que más experiencia da de una mazmorra.

In [16]:
mazmorra = "Bentclock, Laboratory of the Sexy Thieves"
query = """
MATCH (r: Room{dungeon_name: $mazmorra})-[c: CONTAINS]->(m: Monster)
WITH r.room_name as room_name, SUM(c.amount * m.exp) as total_exp
WITH MAX(total_exp) AS max_exp
MATCH (r: Room{dungeon_name: $mazmorra})-[c: CONTAINS]->(m: Monster)
WITH r.room_name as room_name, SUM(c.amount * m.exp) as total_exp, max_exp
WHERE total_exp = max_exp
RETURN room_name, total_exp
"""
results = run_query(query, mazmorra=mazmorra)
results

[{'room_name': 'snobbish catacombs of otaku', 'total_exp': 5450}]